# RAPTOR: Recursive Abstractive Processing and Thematic Organization for Retrieval

Here are good resources to understand the RAPTOR:
- [rag_techniques raptor example](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/raptor.ipynb)
- [RAPTOR official source code](https://github.com/parthsarthi03/raptor)
- My Medium (coming soon...)

First, let's install required libraries and download the document for the demonstration.
 
For the document, here choose the RAPTOR paper itself to demonstrate the RAPTOR model.

In [10]:
!pip install -U arxiv PyMuPDF

In [15]:
import re
import ssl
import tempfile

import arxiv
import fitz

import urllib.error
import urllib.request

In [17]:
def download_paper(url: str) -> str:
    """Downloads the PDF of an ArXiv paper from the given URL and returns its text content."""
    #! Ensure SSL certificate is verified
    try:
        urllib.request.urlopen(url)
    except (ssl.SSLCertVerificationError, urllib.error.URLError):
        ssl._create_default_https_context = ssl._create_unverified_context

    # * Extract ArXiv ID from the URL
    arxiv_match = re.search(r"(\d{4}\.\d{4,5})(v\d+)?", url)
    if not arxiv_match:
        raise ValueError(
            f"Invalid ArXiv URL: {url}. Expected URL should contain ArXiv ID."
        )
    arxiv_id = arxiv_match.group(1)

    # Fetch paper using ArXiv ID
    client = arxiv.Client()
    search_query = arxiv.Search(id_list=[arxiv_id])
    search_result = client.results(search_query)

    try:
        paper = next(search_result)
    except StopIteration:
        raise ValueError(f"Paper not found for ArXiv ID: {arxiv_id}")

    # Download PDF and extract text content
    with tempfile.TemporaryDirectory() as temp_dir:
        pdf_path = paper.download_pdf(dirpath=temp_dir)
        doc = fitz.open(pdf_path)
        text = "".join(page.get_text() for page in doc)

    return text

def chunk_text(text: str, chunk_size: int):...

In [19]:
paper = download_paper("https://arxiv.org/abs/2401.18059")
print(paper[:1000])  # Display first 1000 characters of the paper

Published as a conference paper at ICLR 2024
RAPTOR: RECURSIVE ABSTRACTIVE PROCESSING
FOR TREE-ORGANIZED RETRIEVAL
Parth Sarthi, Salman Abdullah, Aditi Tuli, Shubh Khanna, Anna Goldie, Christopher D. Manning
Stanford University
psarthi@cs.stanford.edu
ABSTRACT
Retrieval-augmented language models can better adapt to changes in world state
and incorporate long-tail knowledge. However, most existing methods retrieve
only short contiguous chunks from a retrieval corpus, limiting holistic under-
standing of the overall document context. We introduce the novel approach of
recursively embedding, clustering, and summarizing chunks of text, constructing
a tree with differing levels of summarization from the bottom up. At inference
time, our RAPTOR model retrieves from this tree, integrating information across
lengthy documents at different levels of abstraction. Controlled experiments show
that retrieval with recursive summaries offers significant improvements over tra-
ditional retrieval-augme